In [ ]:
# load google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
%%shell

pip install cython
# Install pycocotools, the version by default in Colab
# has a bug fixed in https://github.com/cocodataset/cocoapi/pull/354
pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
%%shell

# Download TorchVision repo to use some files from
# references/detection
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/coco_utils.py ../

In [ ]:
%%shell

# download github repo for py file
git clone https://github.com/aliciachenw/11785-project.git
cd 11785-project

cp dataloader.py ../
cp model.py ../
cp train_teacher.py ../
cp coco_evaluate.py ../
cp self_training.py ../
cp train_teacher.py ../

# **Parameters**

In [ ]:
import torch

NUM_WORKERS = 8
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 8
NUM_EPOCHS = 100

TRAINING_RATIO = 0.7

print("use device:", DEVICE)

# **custom dataset**

In [ ]:
from dataloader import FLIRDataset, get_dataloader, split_dataset

# **get dataset and dataloader**

In [ ]:
data_root_dir = '/content/gdrive/MyDrive/FLIR'
dataset = FLIRDataset(data_root_dir + '/train/', 'thermal_annotations.json')

In [ ]:
# split training dataset into labeled and unlabeled
import numpy as np
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from dataloader import collate_fn, split_dataset


# **Get model**

In [ ]:
from model import get_model, save_checkpoint, load_checkpoint

# background and person
num_classes = 2
model = get_model(num_classes, pre_train=True)
model.to(DEVICE)

# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(params, lr=0.001, weight_decay=5e-6)
lr_scheduler = None


# **Train teacher model**

In [ ]:
save_path = '/content/gdrive/MyDrive/FLIR/baseline'
import os
if not os.path.exists(save_path):
    os.mkdir(save_path)

In [ ]:
print(save_path)

In [ ]:
from train_teacher import train_teacher_model

train_teacher_model(model, dataset, optimizer, lr_scheduler, 
                    train_ratio=TRAINING_RATIO, batch_size=BATCH_SIZE, device=DEVICE, 
                    max_epochs=500, print_freq=500, save_path=save_path)

In [ ]:
import numpy as np
np.savez(os.path.join(save_path, "train_loss.npz"), train_loss=np.array(all_training_loss), val_loss=np.array(all_validation_loss))
import matplotlib.pyplot as plt
plt.figure()
plt.plot(all_training_loss)
plt.plot(all_validation_loss)

In [ ]:
# load the best teacher model
checkpoint = '/content/gdrive/MyDrive/FLIR/teacher_model-0.1/0007.tar'
model, optimizer, lr_scheduler, _ = load_checkpoint(model, optimizer, lr_scheduler, DEVICE,checkpoint)

In [ ]:
test_dataset = FLIRDataset(data_root_dir + '/val/', 'thermal_annotations.json')
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

In [ ]:
from coco_evaluate import coco_evaluate
coco_evaluate(model, test_dataloader, DEVICE)

In [ ]:
# visualize the performance
import random
model.eval()
image, target = test_dataset.__getitem__(random.randint(0, test_dataset.__len__() - 1))
predict = model([image.to(DEVICE)])
predict = predict[0]

import matplotlib.pyplot as plt
import matplotlib.patches as patches
image = image[0, :, :].to('cpu').detach().numpy()
fig, ax = plt.subplots(1)
box = predict["boxes"]
index = predict["scores"] > 0.5
box = box[index, :].to('cpu').detach().numpy()
ax.imshow(image, cmap='gray')
for i in range(box.shape[0]):
  rect = patches.Rectangle((box[i, 0], box[i, 1]), box[i, 2] - box[i, 0], box[i, 3] - box[i, 1], linewidth=1, edgecolor='r', facecolor='none')
  ax.add_patch(rect)
print(target["boxes"])

plt.show()
print(predict["scores"][index])
